In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import matplotlib.pyplot as plt
import lib.cell as cell
import lib.spec as spec
from lib.model import NModel
import numpy as np



(train_validate_images, train_validate_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

train_images ,validate_images = np.split(train_validate_images, [40000])
train_labels ,validate_labels = np.split(train_validate_labels, [40000])

# Normalize pixel values to be between 0 and 1
train_images, validate_images, test_images = train_images / 255.0, validate_images / 255.0,test_images / 255.0

In [12]:

cf = spec.Spec.get_configuration_space().sample_configuration()
s = spec.Spec(cf)
while s.valid_spec == False:
    cf = spec.Spec.get_configuration_space().sample_configuration()
    s = spec.Spec(cf)

print(s.matrix)
print(s.ops)

[[0 1 1 0 0 0]
 [0 0 1 0 1 0]
 [0 0 0 1 1 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]]
['input', 'conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3', 'conv1x1-bn-relu', 'output']


In [13]:
i = tf.keras.Input(shape=(32, 32, 3))
nModel = NModel(spec=s, inputs=i)
model = nModel.build()
model.summary()

___________________
re_lu_22 (ReLU)                 (None, 16, 16, 128)  0           batch_normalization_22[0][0]     
__________________________________________________________________________________________________
add_8 (Add)                     (None, 16, 16, 128)  0           re_lu_21[0][0]                   
                                                                 re_lu_22[0][0]                   
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 16, 16, 128)  16384       add_8[0][0]                      
__________________________________________________________________________________________________
batch_normalization_23 (BatchNo (None, 16, 16, 128)  512         conv2d_24[0][0]                  
__________________________________________________________________________________________________
re_lu_23 (ReLU)                 (None, 16, 16, 128)  0           batch_normalization_23[0

In [14]:
optimizer = tf.keras.optimizers.RMSprop(
            learning_rate=tf.keras.experimental.CosineDecay(0.2, 256, alpha=0.0),
            momentum=0.9,
            epsilon=1.0)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=4, batch_size=256, validation_data=(validate_images, validate_labels))

Train on 40000 samples, validate on 10000 samples
Epoch 1/4
40000/40000 [==============================] - 42s 1ms/sample - loss: 1.8895 - accuracy: 0.3231 - val_loss: 107566.1637 - val_accuracy: 0.1027
Epoch 2/4
40000/40000 [==============================] - 37s 936us/sample - loss: 1.4496 - accuracy: 0.4620 - val_loss: 192.5128 - val_accuracy: 0.1282
Epoch 3/4
40000/40000 [==============================] - 37s 937us/sample - loss: 1.4029 - accuracy: 0.4816 - val_loss: 17.1741 - val_accuracy: 0.1211
Epoch 4/4
40000/40000 [==============================] - 38s 939us/sample - loss: 1.4030 - accuracy: 0.4800 - val_loss: 3.8491 - val_accuracy: 0.1787


In [15]:
history.history

{'loss': [1.8895489448547362,
  1.4495863836288452,
  1.402880509185791,
  1.402955818748474],
 'accuracy': [0.3231, 0.46195, 0.4816, 0.480025],
 'val_loss': [107566.1637,
  192.51275043945313,
  17.174112457275392,
  3.8490982215881346],
 'val_accuracy': [0.1027, 0.1282, 0.1211, 0.1787]}

In [16]:
model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 4s 403us/sample - loss: 3.9102 - accuracy: 0.1745


[3.9102051452636717, 0.1745]

In [17]:
print(model.count_params())
np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_variables])

3742346


3728906

In [18]:
model.history.params

{'batch_size': 256,
 'epochs': 4,
 'steps': 157,
 'samples': 40000,
 'verbose': 0,
 'do_validation': True,
 'metrics': ['loss', 'accuracy', 'val_loss', 'val_accuracy']}

In [22]:
model.history.history

{'loss': [1.4638422157287598,
  1.1612712270736694,
  1.1123817365646362,
  1.1109454036712647],
 'accuracy': [0.4653, 0.57965, 0.59985, 0.6013],
 'val_loss': [3.754738680267334,
  7.283068478393555,
  3.8965704246521,
  2.236551708984375],
 'val_accuracy': [0.1128, 0.1562, 0.27, 0.3833]}

In [20]:
model.history.history["accuracy"][-1]

0.480025

In [21]:
data = nModel.train_and_evaluate()
data

Train on 40000 samples, validate on 10000 samples
Epoch 1/4
40000/40000 [==============================] - 40s 992us/sample - loss: 1.4638 - accuracy: 0.4653 - val_loss: 3.7547 - val_accuracy: 0.1128
Epoch 2/4
40000/40000 [==============================] - 36s 895us/sample - loss: 1.1613 - accuracy: 0.5796 - val_loss: 7.2831 - val_accuracy: 0.1562
Epoch 3/4
40000/40000 [==============================] - 36s 899us/sample - loss: 1.1124 - accuracy: 0.5998 - val_loss: 3.8966 - val_accuracy: 0.2700
Epoch 4/4
10000/10000 [==============================] - 4s 391us/sample - loss: 2.2466 - accuracy: 0.3829


{'trainable_parameters': 3728906,
 'params': {'batch_size': 256,
  'epochs': 4,
  'steps': 157,
  'samples': 40000,
  'verbose': 0,
  'do_validation': True,
  'metrics': ['loss', 'accuracy', 'val_loss', 'val_accuracy']},
 'history': {'loss': [1.4638422157287598,
   1.1612712270736694,
   1.1123817365646362,
   1.1109454036712647],
  'accuracy': [0.4653, 0.57965, 0.59985, 0.6013],
  'val_loss': [3.754738680267334,
   7.283068478393555,
   3.8965704246521,
   2.236551708984375],
  'val_accuracy': [0.1128, 0.1562, 0.27, 0.3833]},
 'training_time': 147.2608449459076,
 'train_accuracy': 0.6013,
 'validation_accuracy': 0.3833,
 'test_accuracy': 0.3829}